In [1]:
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
import time
import math
from rt2_assignment1.srv import Command
import actionlib
import rt2_assignment1.msg
from geometry_msgs.msg import Twist
from matplotlib import animation, rc
import ipywidgets as widgets
from IPython.display import display
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation
from nav_msgs.msg import Odometry
from rt2_assignment1.msg import GoalReachingActionResult
%matplotlib widget

rospy.init_node('user_interface')

clock = time.time() #get the offset of time
time1 = list(range(0, 10)) #list for plot the time coordinate 
odom_linear =list() #list to plot the linear velocity gets by /odom
cmdvel_linear=list() #list to plot the linear velocity gets by /cmd_vel
odom_angular =list() #list to plot the angular velocity gets by /odom
cmdvel_angular=list() #list to plot the angular velocity gets by /cmd_vel

def clb_cmdvel(msg):
    global cmdvel_angular,cmdvel_linear
    if len(cmdvel_linear)>10:
      cmdvel_linear.pop(0)
      cmdvel_angular.pop(0)
    #get values
    iupsilon1 = np.float32(msg.linear.x)
    iupsilon2 = np.float32(msg.angular.z)
    #update arrays
    cmdvel_angular.append(iupsilon2)
    cmdvel_linear.append(iupsilon1)

def clb_odom(msg):
    global dx,dy
    global  y_data,x_data
    global time1,odom_angular,odom_linear
    if len(time1)>10:
      time1.pop(0)
      odom_angular.pop(0)
      odom_linear.pop(0)
    #get angular and linear velocity
    y1 = np.float32(msg.twist.twist.angular.z)
    y2 = np.float32(msg.twist.twist.linear.x)
    #update timeline
    t = np.float32(time.time()-clock)
    #actual data inserted in array
    odom_angular.append(y1)
    time1.append(t)
    odom_linear.append(y2)
    
odom_sub=rospy.Subscriber('/odom',Odometry, clb_odom)
cmd_sub=rospy.Subscriber('/cmd_vel', Twist, clb_cmdvel)
    
def init1():
    line1.set_data([], [])
    line2.set_data([], [])
    return [line1,line2]

fig1, ax1 = plt.subplots()

line1, = ax1.plot([], [], color ='r', label ='/odom')
line2, = ax1.plot([], [], color ='b', label ='/cmd_vel')
ax1.set_xlabel('time (s)') 
ax1.set_ylabel('velocity m/s')
ax1.set_title("Linear velocity")
plt.legend(loc='upper right')

def animate1(i):
    #updated values are set in the line
    line1.set_data(time1, odom_linear)
    line2.set_data(time1, cmdvel_linear)
    ax1.axis([time1[0]-0.2,time1[9]+0.2,-0.5,0.5]) #x axis is continously updated too   
    return [line1,line2]

#animator
anim1 = animation.FuncAnimation(fig1, animate1, init_func=init1,
                               frames=100, interval=20, blit=True)

def init2():
    line3.set_data([], [])
    line4.set_data([], [])
    return [line3,line4]

fig2, ax2 = plt.subplots()

line3, = ax2.plot([], [], color ='r', label ='/odom')
line4, = ax2.plot([], [], color ='b', label ='/cmd_vel')
ax2.set_xlabel('time (s)') 
ax2.set_ylabel('velocity rad/s')
ax2.set_title("Angular velocity")
plt.legend(loc='upper right')

def animate2(i):
    line3.set_data(time1, odom_angular)
    line4.set_data(time1, cmdvel_angular)
    ax2.axis([time1[0]-0.2,time1[9]+0.2,-0.5,0.5])
    return [line3,line4]

anim2 = animation.FuncAnimation(fig2, animate2, init_func=init2,
                               frames=100, interval=20, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …